In [3]:
#IMPORTS

import cv2
from ultralytics import YOLO
import requests
import numpy as np

In [2]:
#MODEL INITIALIZATION

model = YOLO("DetectionWeights.pt")

In [ ]:
#MODEL LABEL CHECK

model.names

{0: 'center', 1: 'hole', 2: 'screw'}

In [ ]:
# WEBCAM CODE

def check_similar(given, trial_data):

    x = given[0]
    y = given[1]

    for trial in trial_data:

        trial_x = trial[0]
        trial_y = trial[1]

        if abs(x - trial_x) < 150 and abs(y - trial_y) < 150:
            return True
    
    return False


def live_detection(src, model):
    cap = cv2.VideoCapture(src)

    while cap.isOpened():

        ret, frame = cap.read()

        if not ret:
            break

        results = model(frame, conf=0.65)
        annotated_frame = results[0].plot()

        cv2.imshow("YOLO Inference", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


def make_image(center, holes, screws, frame):

    for elem in ([center], holes, screws):

        color = [(0, 255, 0), (255, 0, 0), (0, 0, 255)][([center], holes, screws).index(elem)]

        for data in elem:

            x, y, w, h = data
            cv2.rectangle(frame, (x - w//2, y - h//2), (x + w//2, y + h//2), color, 2)
    
    return frame


def get_data(center_id, hole_id, screw_id, src, model):

    cap = cv2.VideoCapture(src)

    while cap.isOpened():

        ret, frame = cap.read()

        if not ret:
            break

        results = model(frame, conf=0.65)

        center_data = None
        hole_data = []
        screw_data = []

        for detections in results[0].boxes:
            data = list(map(int, detections.xywh.tolist()[0]))
            item_class = detections.cls[0].item()

            if item_class == center_id:
                center_data = data

            elif item_class == hole_id:
                hole_data.append(data)

            elif item_class == screw_id:
                screw_data.append(data)

        copy_list = []

        for screw in screw_data:
            if not check_similar(screw, hole_data):
                
                copy_list.append(screw)
        
        screw_data = copy_list

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if center_data is not None and len(hole_data) > 0 and len(screw_data) > 0:
            img = make_image(center_data, hole_data, screw_data, frame.copy())
            cv2.imwrite(f"Result.jpg", img)
            cap.release()
            cv2.destroyAllWindows()
            return center_data, hole_data, screw_data

In [ ]:
#WEBCAM CODE

print(get_data(0, 1, 2, 0, model))


0: 480x640 1 center, 3 holes, 3 screws, 46.0ms
Speed: 2.0ms preprocess, 46.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
([250, 266, 20, 24], [[174, 308, 14, 16], [108, 239, 15, 16], [176, 241, 14, 16]], [[404, 189, 13, 15], [401, 236, 12, 15]])


In [6]:
#ESP FUNCTIONALITY CHECK

cam_url = 'http://192.168.107.228/capture'

response = requests.get(cam_url)
if response.status_code == 200:
    # Save the image as a JPEG file
    with open("captured_image.jpg", "wb") as img_file:
        img_file.write(response.content)
    print("Image saved successfully!")
else:
    print(f"Failed to capture image. Status code: {response.status_code}")

Image saved successfully!


In [26]:
#ESP CODE

def check_similar(given, trial_data):

    x = given[0]
    y = given[1]

    for trial in trial_data:

        trial_x = trial[0]
        trial_y = trial[1]

        if abs(x - trial_x) < 150 and abs(y - trial_y) < 150:
            return True
    
    return False

def fetch_image_from_esp(url):
    response = requests.get(url)
    img_array = np.array(bytearray(response.content), dtype=np.uint8)
    img = cv2.imdecode(img_array, -1)
    return img


def make_image(center, holes, screws, frame):

    for elem in ([center], holes, screws):

        color = [(0, 255, 0), (255, 0, 0), (0, 0, 255)][([center], holes, screws).index(elem)]

        for data in elem:

            x, y, w, h = data
            cv2.rectangle(frame, (x - w//2, y - h//2), (x + w//2, y + h//2), color, 2)
    
    return frame


def get_data(center_id, hole_id, screw_id, url, model):



    while True:

        frame = fetch_image_from_esp(url)

        if frame is None:
            break

        results = model(frame, conf=0.65)

        center_data = None
        hole_data = []
        screw_data = []

        for detections in results[0].boxes:
            data = list(map(int, detections.xywh.tolist()[0]))
            item_class = detections.cls[0].item()

            if item_class == center_id:
                center_data = data

            elif item_class == hole_id:
                hole_data.append(data)

            elif item_class == screw_id:
                screw_data.append(data)

        copy_list = []

        for screw in screw_data:
            if not check_similar(screw, hole_data):
                
                copy_list.append(screw)
        
        screw_data = copy_list

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if center_data is not None and len(hole_data) > 0 and len(screw_data) > 0:
            img = make_image(center_data, hole_data, screw_data, frame.copy())
            cv2.imwrite(f"Result.jpg", img)
            cv2.destroyAllWindows()
            return center_data, hole_data, screw_data

In [27]:
#ESP CODE

model = YOLO("DetectionWeights.pt")
cam_url = 'http://192.168.107.228/capture'
print(get_data(0, 1, 2, cam_url, model))



0: 480x640 1 center, 3 screws, 92.0ms
Speed: 6.0ms preprocess, 92.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 screw, 100.4ms
Speed: 3.0ms preprocess, 100.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 196.7ms
Speed: 2.0ms preprocess, 196.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 screw, 132.4ms
Speed: 4.0ms preprocess, 132.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 screw, 84.0ms
Speed: 2.0ms preprocess, 84.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 83.0ms
Speed: 2.0ms preprocess, 83.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 screw, 103.0ms
Speed: 2.0ms preprocess, 103.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 screw, 120.7ms
Speed: 3.0ms preprocess, 120.7ms inference, 2.0ms postprocess

In [ ]:
#SIMPLIFIED VERSION


positions = [(240, 240), (240, 270), (240, 300), (240, 330)]

def check_similar(given, trial_data):

    x = given[0]
    y = given[1]

    trial_x = trial_data[0]
    trial_y = trial_data[1]

    return abs(x - trial_x) < 10 and abs(y - trial_y) < 10

    

def fetch_image_from_esp(url):
    response = requests.get(url)
    img_array = np.array(bytearray(response.content), dtype=np.uint8)
    img = cv2.imdecode(img_array, -1)
    return img

def check_screws(positions, screw_id, url, model):

    for _ in range(10):

        frame = fetch_image_from_esp(url)

        if frame is None:
            break

        results = model(frame, conf=0.65)

        screw_data = []
        result = [False, False, False, False]

        for detections in results[0].boxes:
            data = list(map(int, detections.xywh.tolist()[0]))
            item_class = detections.cls[0].item()

            if item_class == screw_id:
                screw_data.append(data)

        for ind, check in enumerate(result):

            if check:
                continue

            result[ind] = any(check_similar(screw, positions[ind]) for screw in screw_data)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    return result

In [23]:
model = YOLO("DetectionWeights.pt")
cam_url = 'http://192.168.107.228/capture'
print(check_screws(positions, 2, cam_url, model))


0: 480x640 1 center, 1 screw, 93.3ms
Speed: 4.0ms preprocess, 93.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
[[459, 237, 11, 12]] (240, 240)
[[459, 237, 11, 12]] (240, 270)
[[459, 237, 11, 12]] (240, 300)
[[459, 237, 11, 12]] (240, 330)

0: 480x640 2 screws, 182.7ms
Speed: 6.8ms preprocess, 182.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
[[455, 238, 11, 12], [459, 266, 9, 13]] (240, 240)
[[455, 238, 11, 12], [459, 266, 9, 13]] (240, 270)
[[455, 238, 11, 12], [459, 266, 9, 13]] (240, 300)
[[455, 238, 11, 12], [459, 266, 9, 13]] (240, 330)

0: 480x640 2 screws, 92.0ms
Speed: 2.0ms preprocess, 92.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
[[454, 237, 11, 12], [459, 266, 9, 12]] (240, 240)
[[454, 237, 11, 12], [459, 266, 9, 12]] (240, 270)
[[454, 237, 11, 12], [459, 266, 9, 12]] (240, 300)
[[454, 237, 11, 12], [459, 266, 9, 12]] (240, 330)

0: 480x640 1 center, 2 screws, 87.6ms
Speed: 1.8ms preprocess, 87.6ms infe